In [2]:
import warnings
warnings.filterwarnings('ignore')

path = "../../kaggle/data/"

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

#main module for evaluation
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, mean_squared_error,r2_score

def calcMetrics(testActualVal, predictions):
    #regression evaluation measures
    data={"RMSLE":[mean_squared_log_error(testActualVal, predictions)**0.5],
         "MAE":[mean_absolute_error(testActualVal, predictions)],
         "RMSE":[mean_squared_error(testActualVal, predictions)**0.5],
         "R2":[r2_score(testActualVal, predictions)]}
    metric_df=pd.DataFrame(data)
    return metric_df

def split(df, useful_features, target):
    val_start_index=df.shape[0]-len(df["date"][df["date"] >=pd.to_datetime("2017-01-01")])
    train = df[:val_start_index]
    val=df[val_start_index:]
    X_train = train[useful_features]
    y_train = train[target]
    X_valid = val[useful_features]
    y_valid = val[target]
    return X_train, y_train, X_valid, y_valid

def pl(df_compare, actual, pred):
    fig, ax = plt.subplots(figsize=(14,3))
    df = df_compare[df_compare.date >= pd.to_datetime("2017-01-02")]
    g1=sns.lineplot(data=df, x = "date", y = actual, ax = ax)
    g2=sns.lineplot(data=df, x = "date", y = pred, ax = ax)
    g1.set(yscale='log')
    g2.set(yscale='log')
    plt.show()
    plt.close()

## Chargement de Dataset prétraité

In [3]:
train_merged = pd.read_csv(path + "train_merged.csv")
train_merged["date"] = pd.to_datetime(train_merged.date)

np.array(train_merged.columns)

array(['date', 'store_nbr', 'family', 'onpromotion', 'sales', 'city',
       'state', 'typestores', 'cluster', 'typeholiday', 'locale',
       'locale_name', 'description', 'transferred', 'dcoilwtico',
       'day_of_week', 'month', 'year'], dtype=object)

## Vérifications de cohérence des données

In [3]:
# vérification de l'ordre des valeurs 'family'
nbr_family = len(list(train_merged['family'].value_counts().index))
total1 = 0

for j in range(nbr_family):
    f_val = train_merged.iloc[j ,2]
    
    for i in range(int(train_merged.shape[0] / nbr_family)) :
        if f_val != train_merged.iloc[j + i * nbr_family, 2] :
            total1 += 1
            
print('nbr_family : ', nbr_family)
print(total1)

nbr_stores = len(list(train_merged['store_nbr'].value_counts().index))
nbr_rows = train_merged.shape[0]
nbr_jours = int( nbr_rows / nbr_stores / nbr_family)

print('nbr rows', nbr_rows)
print('nbr_jours', nbr_jours)
print('nbr_stores', nbr_stores)
print('nbr_family', nbr_family)

# vérification de l'ordre des valeurs 'nbr_stores'

total2 = 0

for j in range(nbr_jours):
    for s in range(nbr_stores):
        s_val = train_merged.iloc[s* nbr_family ,1]
        for f in range(nbr_family):
            if s_val != train_merged.iloc[ j * nbr_family * nbr_stores + s * nbr_family + f ,1]:
                total2 += 1

print('nbr_stores : ', nbr_stores)
print(total2)

nbr_family :  33
0
nbr rows 3029400
nbr_jours 1700
nbr_stores 54
nbr_family 33
nbr_stores :  54
0


## Ajout de variables numériques : 15 valeurs antérieures de sales par ligne
    un délai de 16 jours est à respecter, pour prendre en compte 15 jours successifs à prédire 

In [4]:
# séparation des données en 1782 Datasets, ensuite enrichir les données avec 15 valeurs antérieus de sales

def get_df(store_nbr, family):
    df_auto_str1 = train_merged[ (train_merged.family == family) & (train_merged.store_nbr == store_nbr)  ][["date", "sales"]]
    columns = []
    # création de variables prédectives en remontant jusqu'à la valeur de sales de 30 jours auparavant.
    # on ne va pas prendre les valeurs des 15 derniers jours vu qu'on ne dispose pas de ces données pour le Dataset de test
    for j in range(15, 30):
        col = "J-{}".format(str(j+1))
        columns.append(col)
        df_auto_str1[col] = df_auto_str1.sales.shift(j+1)
    df_auto_str1 = df_auto_str1[["date"] + columns + ["sales"]]
    return df_auto_str1

dfs = [get_df(i+1, family) for i in range(54) for family in [v[0] for v in train_merged[:33][['family']].values]] 
print(len(dfs)) 
print(33*54)

1782
1782


In [5]:
dfs[1000].tail(31)

,date,J-16,J-17,J-18,J-19,J-20,J-21,J-22,J-23,J-24,J-25,J-26,J-27,J-28,J-29,J-30,sales
2974960,2017-08-01,312.0,259.0,312.0,184.0,204.0,313.0,679.0,333.0,280.0,214.0,235.0,178.0,274.0,330.0,525.0,504.0
2976742,2017-08-02,387.0,312.0,259.0,312.0,184.0,204.0,313.0,679.0,333.0,280.0,214.0,235.0,178.0,274.0,330.0,216.0
2978524,2017-08-03,311.0,387.0,312.0,259.0,312.0,184.0,204.0,313.0,679.0,333.0,280.0,214.0,235.0,178.0,274.0,288.0
2980306,2017-08-04,193.0,311.0,387.0,312.0,259.0,312.0,184.0,204.0,313.0,679.0,333.0,280.0,214.0,235.0,178.0,331.0
2982088,2017-08-05,187.0,193.0,311.0,387.0,312.0,259.0,312.0,184.0,204.0,313.0,679.0,333.0,280.0,214.0,235.0,242.0
2983870,2017-08-06,324.0,187.0,193.0,311.0,387.0,312.0,259.0,312.0,184.0,204.0,313.0,679.0,333.0,280.0,214.0,364.0
2985652,2017-08-07,188.0,324.0,187.0,193.0,311.0,387.0,312.0,259.0,312.0,184.0,204.0,313.0,679.0,333.0,280.0,409.0
2987434,2017-08-08,424.0,188.0,324.0,187.0,193.0,311.0,387.0,312.0,259.0,312.0,184.0,204.0,313.0,679.0,333.0,341.0
2989216,2017-08-09,582.0,424.0,188.0,324.0,187.0,193.0,311.0,387.0,312.0,259.0,312.0,184.0,204.0,313.0,679.0,237.0
2990998,2017-08-10,332.0,582.0,424.0,188.0,324.0,187.0,193.0,311.0,387.0,312.0,259.0,312.0,184.0,204.0,313.0,164.0


#### Création d'un Dataset consolidé

In [6]:
# reclassement des colonnes 
columns_merged_tr =  ['date'] + list(train_merged.drop(['date', 'sales'], axis=1).columns) + ['sales']
df = train_merged[columns_merged_tr]

# ajout des variables créées dans les dans le Dataset
shifted_days = ['J-{}'.format(j+1) for j in range(15, 30)]
df[shifted_days] = np.nan

df.tail(2)

,date,store_nbr,family,onpromotion,city,state,typestores,cluster,typeholiday,locale,...,J-21,J-22,J-23,J-24,J-25,J-26,J-27,J-28,J-29,J-30
3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,Quito,Pichincha,B,6,NDay,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3029399,2017-08-31,9,SEAFOOD,0,Quito,Pichincha,B,6,NDay,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# concaténation des dataframes créées et ensuite remettre les lignes dans le même ordre initial 
result_df = pd.concat(dfs)
result_df = result_df.sort_index()
print(result_df.shape)

(3029400, 17)


In [9]:
# vérification 

index = 290741
print(result_df.sales.iloc[index])
print(df.sales.iloc[index])

37.0
37.0


In [10]:
# fusion des deux datasets 
df[shifted_days] = result_df[shifted_days]

# suppression des premies jours avec NAN
df2 = df[33*54*30:]

In [11]:
df.iloc[33*54*29]

date           2013-01-30 00:00:00
store_nbr                        1
family                  AUTOMOTIVE
onpromotion                      0
city                         Quito
state                    Pichincha
typestores                       D
cluster                         13
typeholiday                   NDay
locale                         NaN
locale_name                    NaN
description                    NaN
transferred                    NaN
dcoilwtico                   97.98
day_of_week                      3
month                            1
year                          2013
sales                          6.0
J-16                           2.0
J-17                           2.0
J-18                           2.0
J-19                           3.0
J-20                           2.0
J-21                           2.0
J-22                           2.0
J-23                           0.0
J-24                           2.0
J-25                           5.0
J-26                

In [12]:
df.iloc[33*54*30]

date           2013-01-31 00:00:00
store_nbr                        1
family                  AUTOMOTIVE
onpromotion                      0
city                         Quito
state                    Pichincha
typestores                       D
cluster                         13
typeholiday                   NDay
locale                         NaN
locale_name                    NaN
description                    NaN
transferred                    NaN
dcoilwtico                   97.65
day_of_week                      4
month                            1
year                          2013
sales                          0.0
J-16                           1.0
J-17                           2.0
J-18                           2.0
J-19                           2.0
J-20                           3.0
J-21                           2.0
J-22                           2.0
J-23                           2.0
J-24                           0.0
J-25                           2.0
J-26                

In [13]:
df2.isna().sum()

date                 0
store_nbr            0
family               0
onpromotion          0
city                 0
state                0
typestores           0
cluster              0
typeholiday          0
locale         2530440
locale_name    2530440
description    2530440
transferred    2530440
dcoilwtico           0
day_of_week          0
month                0
year                 0
sales            28512
J-16                 0
J-17                 0
J-18                 0
J-19                 0
J-20                 0
J-21                 0
J-22                 0
J-23                 0
J-24                 0
J-25                 0
J-26                 0
J-27                 0
J-28                 0
J-29                 0
J-30                 0
dtype: int64

In [14]:
np.array(df2.columns)

array(['date', 'store_nbr', 'family', 'onpromotion', 'city', 'state',
       'typestores', 'cluster', 'typeholiday', 'locale', 'locale_name',
       'description', 'transferred', 'dcoilwtico', 'day_of_week', 'month',
       'year', 'sales', 'J-16', 'J-17', 'J-18', 'J-19', 'J-20', 'J-21',
       'J-22', 'J-23', 'J-24', 'J-25', 'J-26', 'J-27', 'J-28', 'J-29',
       'J-30'], dtype=object)

## Selection des feautures

In [15]:
useful_features=['date', 'store_nbr', 'family', 'onpromotion', 'typeholiday', 'dcoilwtico', 'city', 'state', 'typestores',
                 'cluster', 'day_of_week', 'month', 'year', 'sales', 'J-16', 'J-17', 'J-18', 'J-19', 'J-20', 'J-21', 'J-22', 'J-23',
                 'J-24', 'J-25', 'J-26', 'J-27', 'J-28', 'J-29', 'J-30']
                 
category_columns=['store_nbr', 'family', 'typeholiday', 'city', 'state', 'typestores', 'cluster']
                 
                    
time_columns=['onpromotion', 'dcoilwtico', 'month', 'day_of_week', 'year',
              'J-16', 'J-17', 'J-18', 'J-19', 'J-20', 'J-21', 'J-22', 'J-23', 'J-24', 'J-25', 'J-26', 'J-27', 'J-28', 'J-29', 'J-30']

# names of columns to train xgb
col_names_classic_ml=['store_nbr', 'family', 'onpromotion', 'typeholiday', 'dcoilwtico', 'city', 'state', 'typestores',
                      'cluster', 'day_of_week', 'month', 'year', 'J-16', 'J-17', 'J-18', 'J-19', 'J-20', 'J-21', 'J-22', 'J-23',
                      'J-24', 'J-25', 'J-26', 'J-27', 'J-28', 'J-29', 'J-30']

# names of columns after pipeline transformations, 
# note ordering of this list isn't arbitrary.
# I manually adjusted ordering after getting feature names out of pipeline and verfiying ordering 
col_names_classic_ml_transformed=['store_nbr', 'family', 'typeholiday', 'city', 'state', 'typestores', 'cluster',
       'day_of_week_sin','day_of_week_cos', 'month_sin', 'month_cos','year_sin', 'year_cos',
       'day_of_week_sin day_of_week_cos','day_of_week_sin month_sin', 'day_of_week_sin month_cos',
       'day_of_week_sin year_sin', 'day_of_week_sin year_cos','day_of_week_cos month_sin',
       'day_of_week_cos month_cos','day_of_week_cos year_sin', 'day_of_week_cos year_cos','month_sin month_cos',
       'month_sin year_sin','month_sin year_cos','month_cos year_sin','month_cos year_cos', 'year_sin year_cos',
       'onpromotion', 'dcoilwtico', 'J-16', 'J-17', 'J-18', 'J-19', 'J-20', 'J-21', 'J-22', 'J-23', 'J-24', 'J-25', 
       'J-26', 'J-27', 'J-28', 'J-29', 'J-30']

In [16]:
col_names_classic_ml_transformed

['store_nbr',
 'family',
 'typeholiday',
 'city',
 'state',
 'typestores',
 'cluster',
 'day_of_week_sin',
 'day_of_week_cos',
 'month_sin',
 'month_cos',
 'year_sin',
 'year_cos',
 'day_of_week_sin day_of_week_cos',
 'day_of_week_sin month_sin',
 'day_of_week_sin month_cos',
 'day_of_week_sin year_sin',
 'day_of_week_sin year_cos',
 'day_of_week_cos month_sin',
 'day_of_week_cos month_cos',
 'day_of_week_cos year_sin',
 'day_of_week_cos year_cos',
 'month_sin month_cos',
 'month_sin year_sin',
 'month_sin year_cos',
 'month_cos year_sin',
 'month_cos year_cos',
 'year_sin year_cos',
 'onpromotion',
 'dcoilwtico',
 'J-16',
 'J-17',
 'J-18',
 'J-19',
 'J-20',
 'J-21',
 'J-22',
 'J-23',
 'J-24',
 'J-25',
 'J-26',
 'J-27',
 'J-28',
 'J-29',
 'J-30']

In [17]:
# selection des feautures
df3 = df2[useful_features]

# Cast en string des variables catégoriales
for column in category_columns:
    df3[column] = df3[column].astype('str')

# Cast en float des variables numériques
for column in time_columns:
    df3[column] = df3[column].astype('float')

## Split en données de training, de validation et de test

In [18]:
# données de test pour la prédiction de submissoin
X_test = df.drop(['date', 'sales'], axis=1)[df.index > 3000887]

# données de train, split en données d'entrainement et donnée de validation
X_train, y_train, X_valid, y_valid =split(df[df.index < 3000888], list(df.drop(['date', 'sales'], axis=1)), 'sales')

print(X_test.shape)
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(y_valid.shape[0] / y_train.shape[0])

(28512, 31)
(2596374, 31)
(2596374,)
(404514, 31)
(404514,)
0.15579958819492107


## Numérisation des données catégorielles et scalarisatrion

In [19]:
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.preprocessing import TargetEncoder
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer, KBinsDiscretizer

category_feat = Pipeline(steps=[("target_encode", TargetEncoder(target_type="continuous"))])

# helper functions to be able to get feature names out of functional transformer 
def f_out_sin(self,input_features):
    return input_features
def f_out_cos(self,input_features):
    return input_features
    
# functions to transform time features with sine cosine transformation 
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi),feature_names_out=f_out_sin)

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi), feature_names_out=f_out_cos)


#adding polynomial transformation on sine_cosine transformed time features to capture linear interactions between time features
time_feat = make_pipeline(
                        ColumnTransformer([
                            #("cyclic_day_of_week", periodic_spline_transformer(7, n_splines=3), ["day_of_week"]),
                            ("day_of_week_sin", sin_transformer(7), ["day_of_week"]),
                            ("day_of_week_cos", cos_transformer(7), ["day_of_week"]),
                            #("cyclic_month", periodic_spline_transformer(12, n_splines=6), ["month"]),
                            ("month_sin", sin_transformer(12), ["month"]),
                            ("month_cos", cos_transformer(12), ["month"]),
                            ("year_sin", sin_transformer(365), ["year"]),
                            ("year_cos", cos_transformer(365), ["year"]),   
                            ],remainder='drop'),
    #Nystroem(kernel="poly", degree=2,n_jobs=-1, n_components=85, random_state=0),
    PolynomialFeatures(degree=2, interaction_only=True, include_bias=False))


preprocess_pipe = Pipeline(steps=[
    ('encoder', ColumnTransformer(
                    transformers=[
                        ("category_trans",category_feat, category_columns),
                        ("time_trans",time_feat,["day_of_week","month","year"] ),
                                ],
                                remainder="passthrough", verbose_feature_names_out=True
                            )),
    ('scaler', MinMaxScaler()),
    ("pandarizer2", FunctionTransformer(lambda x: pd.DataFrame(x, columns =  col_names_classic_ml_transformed)))
                            ],verbose = True)

In [20]:
preprocess_pipe.fit(X_train[col_names_classic_ml], y_train)

[Pipeline] ........... (step 1 of 3) Processing encoder, total=   3.7s
[Pipeline] ............ (step 2 of 3) Processing scaler, total=   0.4s
[Pipeline] ....... (step 3 of 3) Processing pandarizer2, total=   0.0s


Pipeline(steps=[('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('category_trans',
                                                  Pipeline(steps=[('target_encode',
                                                                   TargetEncoder(target_type='continuous'))]),
                                                  ['store_nbr', 'family',
                                                   'typeholiday', 'city',
                                                   'state', 'typestores',
                                                   'cluster']),
                                                 ('time_trans',
                                                  Pipeline(steps=[('columntransformer',
                                                                   ColumnTransformer(transformers=[('day_of_week_...
                                                                                                    FunctionTransformer(feature_names_out=<function f_out_cos at 0x35e26ef20>,
                                                                                                                        func=<function cos_transformer.<locals>.<lambda> at 0x35e279260>),
                                                                                                    ['year'])])),
                                                                  ('polynomialfeatures',
                                                                   PolynomialFeatures(include_bias=False,
                                                                                      interaction_only=True))]),
                                                  ['day_of_week', 'month',
                                                   'year'])])),
                ('scaler', MinMaxScaler()),
                ('pandarizer2',
                 FunctionTransformer(func=<function <lambda> at 0x35e279300>))],
         verbose=True)

In [21]:
X_train = preprocess_pipe.transform(X_train[col_names_classic_ml])
X_valid = preprocess_pipe.transform(X_valid[col_names_classic_ml])

In [22]:
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance, plot_tree
from sklearn.model_selection import GridSearchCV

#Enabling memory growth for GPU
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
import tensorflow as tf
tf.config.list_physical_devices('GPU') 

# for reproducibility
seed0 = 1337
np.random.seed(seed0) 
tf.keras.utils.set_random_seed(seed0)
tf.config.experimental.enable_op_determinism()
tf.random.set_seed(seed0)

# call back to avoid overfitting
early_stop = xgb.callback.EarlyStopping(rounds=10,
                                        metric_name='rmse',
                                        maximize=False,
                                        save_best= True,
                                        )

### Optimisaion des hperparamètres avec GridSearchCV

In [23]:
# pour une validation croisée avec GridSearch pour selectionner les hyperparamtres optimaux
def grid_search(Model, params, X_tr, y_tr):
    
    gs = GridSearchCV(Model(), 
                    params, 
                    cv=5, 
                    n_jobs=-1, 
                    return_train_score=True, 
                    verbose=1)
    gs.fit(X_tr, y_tr)
    return gs
    
# pour renvoyer la liste des hyperparamètres testés avec gridSearch dans l'ordre décroissant de performance
def results(grid):
    print('best params : {}'.format(grid.best_params_))
    resultats = pd.DataFrame(grid.cv_results_)
    return resultats[[col for col in  resultats.columns if 'split' not in col]].sort_values('rank_test_score')


# pour entrainer le modèle avec les meilleurs paramètres sélectionnés par GridSearch et calcule renvoyer un triplet : 
#liste des valeurs prédites, erreur rmse et score r2
def fit_with_best_params(grid, X_train, y_train, X_test, y_test):
    
    print(grid.best_estimator_)
    
    print()
    
    model = grid.best_estimator_.fit(X_train, y_train)
    prediction = model.predict(X_test)
    rmse = mean_squared_error(prediction, y_test, squared=False)
    print("RMSE : %.6f" %rmse)
    
    score(grid.best_estimator_, X_train, y_train, X_test, y_test)
    r2 = r2_score(y_test, prediction)
    #print("R2 : %.6f" %r2)
    
    return prediction, rmse, r2

#{
#'n_estimators':  [ 300,400,600], 
#'max_depth': [5,6,7], 
#'learning_rate': [0.02, 0.05, 0.1], 
#'gamma': [0.5, 1, 3], 
#'min_child_weight': [1, 3, 5]
#}

In [ ]:
# training xgboost
#xgboost_v00 = XGBRegressor(random_state = seed0,verbosity = 0, n_jobs = -1, reg_lambda = 0.005, learning_rate = 0.01, device = 'gpu',
#            n_estimators = 5000, objective ='reg:squarederror',callbacks = [early_stop])
#xgboost_v00.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], verbose = False)

xgb_grid = GridSearchCV(XGBRegressor(), 
                         {
                           'random_state' : [seed0],
                           'verbosity' : [0], 
                           'n_jobs' : [-1],
                           'reg_lambda': [0.005],
                           'learning_rate': [0.01],
                           'device': ['gpu'],
                           'n_estimators': [5000], 
                           'objective': ['reg:squarederror'],
                           'callbacks': [[early_stop]]
                         },   
                         cv=5, 
                         n_jobs=-1, 
                         return_train_score=True, 
                         verbose=1)

grid = xgb_grid.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], verbose = False)
#pred_xgb_2, rmse_xgb_2, r2_xgb_2 = fit_with_best_params(xgb_grid_2, X2_train, y2_train, X2_test, y2_test)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


## Prédiction et évaluation des données de validation

In [46]:
# prédition des données de test
y_pred_xgb = xgboost_v00.predict(X_valid)

# conversion des valeurs negatives en 0
y_pred_xgb=np.where(y_pred_xgb<0,0,y_pred_xgb)

# calcule de performnce
calcMetrics(y_valid, y_pred_xgb)

,RMSLE,MAE,RMSE,R2
0,0.777008,87.250816,340.508586,0.936867


## Prédiction et évaluation des données de test et création de fichier de submission

In [47]:
# scalarisation de la matrice de test 
X_test = preprocess_pipe.transform(X_test[col_names_classic_ml])

# prédition des données de test
y_pred_test = xgboost_v00.predict(X_test)

# conversion des valeurs negatives en 0
y_pred_test=np.where(y_pred_test<0,0,y_pred_test)

y_pred_test.shape

(28512,)

In [48]:
submission = pd.DataFrame(y_pred_test, columns=['sales'])
submission['id'] = index = df[df.index > 3000887].index
submission[['id', 'sales']]

,id,sales
0,3000888,4.875371
1,3000889,1.755887
2,3000890,34.835281
3,3000891,2246.473633
4,3000892,1.755887
...,...,...
28507,3029395,361.075195
28508,3029396,113.470139
28509,3029397,1283.503052
28510,3029398,108.234444


In [49]:
submission[['id', 'sales']].to_csv(path + "submission.csv", index=False, header=True, sep=',')

**Score Kaggle : 0.7161**